In [ ]:
import pickle
import imageio as iio
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from moviepy.editor import VideoClip
%matplotlib widget

In [ ]:
correctly_synch_vid = "test_data/Server_structure/Calibrations/230214/230214_Ground1_synchronized_downsampled80fps.mp4"
broken_vid = "test_data/Server_structure/Calibrations/230214/230214_Bottom_synchronized_downsampled80fps.mp4"

In [ ]:
broken_led_img = iio.v3.imread(broken_vid, index=0)
correct_led_img = iio.v3.imread(correctly_synch_vid, index=0)

In [ ]:
fig = plt.figure()
plt.imshow(broken_led_img)
plt.show()

In [ ]:
broken_x = int(input("x"))
broken_y = int(input("y"))

In [ ]:
fig = plt.figure()
plt.imshow(correct_led_img)
plt.show()

In [ ]:
correct_x = int(input("x"))
correct_y = int(input("y"))

In [ ]:
broken_reader = iio.v2.get_reader(broken_vid)
broken_timeseries = [broken_reader.get_data(index=i)[broken_y, broken_x].mean() for i in range(broken_reader.count_frames())]
correct_reader = iio.v2.get_reader(correctly_synch_vid)
correct_timeseries = [correct_reader.get_data(index=i)[correct_y, correct_x].mean() for i in range(correct_reader.count_frames())]

In [ ]:
start = 1000
end = 2000
plt.figure()
plt.plot(broken_timeseries[start:end], c="red")
plt.plot(correct_timeseries[start:end], c="green")
plt.show()

In [ ]:
broken_range = (280, 370)

In [ ]:
offset = correct_reader.count_frames() - broken_reader.count_frames()
fps = 80

In [ ]:
output_dir = Path(broken_vid).parent
output_video = output_dir.joinpath(Path(broken_vid).stem + "_framesfixed.mp4")
rename_video = output_dir.joinpath(Path(broken_vid).stem + "_framesbroken.mp4")
black_image = np.zeros(broken_led_img.shape)

In [ ]:
def get_frame(index: int):
    index = round(index*fps)
    if index > broken_range[0]:
        if index < broken_range[1]:
            return black_image
        else:
            frame_index = index - offset
    else:
        frame_index = index
    frame = broken_reader.get_data(frame_index)
    return frame

In [ ]:
out_clip = VideoClip(get_frame, duration=(broken_reader.count_frames()+offset)/80)
out_clip.write_videofile(str(output_video), fps=fps, verbose=True, logger=None)

In [ ]:
Path(broken_vid).rename(rename_video)
Path(output_video).rename(broken_vid)